# Lab 10: Advanced Phishing Email Classifier

Build a comprehensive machine learning classifier to detect diverse phishing attacks.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab10_phishing_classifier.ipynb)

## Learning Objectives
- Multi-class phishing classification (BEC, spear-phishing, credential theft, malware delivery)
- Advanced text preprocessing and feature extraction
- TF-IDF and word embeddings for email analysis
- Header analysis (SPF, DKIM, DMARC)
- URL and attachment risk scoring
- Model evaluation with security-focused metrics
- Adversarial evasion awareness

## Phishing Attack Taxonomy

Modern phishing attacks vary significantly:
1. **Credential Phishing** - Fake login pages, account verification
2. **Business Email Compromise (BEC)** - CEO/CFO impersonation, invoice fraud
3. **Spear Phishing** - Targeted attacks with personalized content
4. **Whaling** - Targeting executives
5. **Vendor Email Compromise (VEC)** - Supply chain fraud
6. **Malware Delivery** - Weaponized attachments, drive-by downloads
7. **Callback Phishing** - Phone-based social engineering

**Next:** Lab 31 (Malware Clustering)

In [ ]:
# Colab: Install dependencies (skip this cell locally - packages already in venv)
# %pip install -q scikit-learn pandas numpy matplotlib seaborn plotly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score

# Plotly for interactive visualizations (works great in Colab)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set matplotlib style (fallback)
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("husl")

# Plotly template optimized for Colab (works in light/dark mode)
PLOTLY_TEMPLATE = "plotly_white"

## 1. Load and Explore Data

In [ ]:
# Comprehensive phishing email dataset with diverse attack types
import random
from typing import List, Dict, Tuple


class PhishingDataGenerator:
    """Generate diverse phishing email samples for training."""

    # Legitimate email templates
    LEGITIMATE_TEMPLATES = [
        # Internal communications
        {
            "subject": "Team meeting tomorrow at 3pm",
            "body": "Hi team, reminder about our weekly sync meeting tomorrow in Conference Room A. Please bring your project updates.",
            "type": "internal",
        },
        {
            "subject": "Q3 report attached",
            "body": "Please find attached the quarterly report for review. Let me know if you have any questions.",
            "type": "internal",
        },
        {
            "subject": "Lunch plans?",
            "body": "Hey! Want to grab lunch today? I was thinking about trying that new place downtown.",
            "type": "internal",
        },
        {
            "subject": "Project deadline extension",
            "body": "Good news - the client agreed to extend the deadline by two weeks. We now have until the 15th.",
            "type": "internal",
        },
        {
            "subject": "Welcome to the team!",
            "body": "Hi Sarah, welcome to the company! Please reach out if you need any help getting started.",
            "type": "internal",
        },
        {
            "subject": "Meeting notes from yesterday",
            "body": "Hi all, please find the meeting notes attached. Action items are highlighted in yellow.",
            "type": "internal",
        },
        {
            "subject": "Vacation request approved",
            "body": "Your time off request for Dec 23-27 has been approved. Enjoy your holiday!",
            "type": "internal",
        },
        {
            "subject": "Expense report submitted",
            "body": "Your expense report #12345 has been submitted for approval. Expected processing time is 3-5 business days.",
            "type": "internal",
        },
        # External legitimate
        {
            "subject": "Your order has shipped",
            "body": "Great news! Your order #ABC123 has shipped and will arrive by Friday. Track your package here: tracking.ups.com/abc123",
            "type": "external",
        },
        {
            "subject": "Invoice #INV-2024-001",
            "body": "Please find attached invoice #INV-2024-001 for services rendered in December. Payment due within 30 days.",
            "type": "external",
        },
        {
            "subject": "Newsletter: January Edition",
            "body": "Welcome to our monthly newsletter! This month we cover industry trends and upcoming events.",
            "type": "external",
        },
        {
            "subject": "Appointment confirmation",
            "body": "This confirms your appointment on January 15th at 2:00 PM. Reply to reschedule if needed.",
            "type": "external",
        },
        {
            "subject": "Thank you for your purchase",
            "body": "Thank you for shopping with us. Your receipt is attached. Questions? Contact support@store.com",
            "type": "external",
        },
    ]

    # Credential phishing templates
    CREDENTIAL_PHISHING = [
        {
            "subject": "Urgent: Your account has been compromised",
            "body": "We detected unusual activity on your account. Verify your identity immediately by clicking here: bit.ly/verify-now or your account will be suspended.",
            "subtype": "credential",
        },
        {
            "subject": "Security Alert: Password reset required",
            "body": "Your password expires today. Update it now to avoid losing access: secure-login.com/reset. This link expires in 24 hours.",
            "subtype": "credential",
        },
        {
            "subject": "Microsoft 365: Verify your account",
            "body": "Your Microsoft 365 subscription needs verification. Please sign in at microsft-verify.com to continue using your services.",
            "subtype": "credential",
        },
        {
            "subject": "Your Google Account: Unusual sign-in prevented",
            "body": "Someone tried to sign in to your account from a new device. If this wasn't you, secure your account at google-security.net",
            "subtype": "credential",
        },
        {
            "subject": "DocuSign: Document waiting for signature",
            "body": "John Smith shared a document with you. Click here to review and sign: docusign-secure.com/doc/abc123",
            "subtype": "credential",
        },
        {
            "subject": "LinkedIn: Please confirm your email",
            "body": "We noticed you haven't verified your email. Complete verification now: linkedln-verify.com/confirm",
            "subtype": "credential",
        },
        {
            "subject": "Apple ID: Your account has been locked",
            "body": "Your Apple ID has been locked due to suspicious activity. Unlock it now: apple-id-support.com/unlock",
            "subtype": "credential",
        },
        {
            "subject": "Netflix: Update your payment information",
            "body": "We couldn't process your payment. Update your billing info to avoid service interruption: netflix-billing.com/update",
            "subtype": "credential",
        },
    ]

    # BEC (Business Email Compromise) templates
    BEC_PHISHING = [
        {
            "subject": "Urgent wire transfer needed",
            "body": "Hi, I need you to process an urgent wire transfer of $45,000 to a new vendor. Please keep this confidential and let me know when done. - CEO",
            "subtype": "bec_ceo_fraud",
        },
        {
            "subject": "Quick favor needed",
            "body": "Are you in the office? I need your help with something urgent but can't call right now. Reply ASAP. - John (CFO)",
            "subtype": "bec_ceo_fraud",
        },
        {
            "subject": "RE: Updated bank details",
            "body": "Please note our bank account has changed. All future payments should go to: Account: 123456789, Routing: 987654321. - Vendor Accounting",
            "subtype": "bec_invoice_fraud",
        },
        {
            "subject": "Invoice Payment - URGENT",
            "body": "The attached invoice is past due. Please process payment immediately to avoid late fees. Our new banking details are included.",
            "subtype": "bec_invoice_fraud",
        },
        {
            "subject": "Payroll update needed",
            "body": "Hi HR, I need to update my direct deposit information before the next payroll. Please change it to account #9876543210.",
            "subtype": "bec_payroll_diversion",
        },
        {
            "subject": "Gift cards needed for client appreciation",
            "body": "I'm in a meeting and can't talk. Please purchase 5 Amazon gift cards ($200 each) for client appreciation. Send me the codes. - Director",
            "subtype": "bec_gift_card",
        },
        {
            "subject": "Confidential acquisition discussion",
            "body": "We're in confidential discussions about acquiring a competitor. I need you to wire $125,000 for the deposit. Keep this between us.",
            "subtype": "bec_ceo_fraud",
        },
    ]

    # Spear phishing templates
    SPEAR_PHISHING = [
        {
            "subject": "Speaking opportunity at Tech Conference 2024",
            "body": "Dear Dr. Smith, we'd like to invite you to speak at our conference. Please review the attached proposal and speaker agreement.",
            "subtype": "spear_personalized",
        },
        {
            "subject": "RE: Your recent publication",
            "body": "I read your paper on machine learning security with great interest. I'd like to discuss collaboration opportunities. Please see attached proposal.",
            "subtype": "spear_personalized",
        },
        {
            "subject": "Your LinkedIn connection request",
            "body": "Hi John, thanks for connecting! I noticed you work at Acme Corp. I have an opportunity that might interest you. Details attached.",
            "subtype": "spear_linkedin",
        },
        {
            "subject": "Alumni network: Job opportunity",
            "body": "Fellow Stanford alum here! Our company has an opening that matches your background. Check out the role description attached.",
            "subtype": "spear_personalized",
        },
        {
            "subject": "Follow-up from today's meeting",
            "body": "Great meeting you at the conference today! As discussed, here's the proposal document. Looking forward to your feedback.",
            "subtype": "spear_personalized",
        },
    ]

    # Malware delivery templates
    MALWARE_PHISHING = [
        {
            "subject": "Invoice #INV-38291 attached",
            "body": "Please find your invoice attached. Enable macros to view the document properly. Contact billing@suspicious.com with questions.",
            "subtype": "malware_invoice",
        },
        {
            "subject": "Your resume was received",
            "body": "Thank you for applying to the position. Please open the attached form to complete your application. Enable content to proceed.",
            "subtype": "malware_job",
        },
        {
            "subject": "Shipping notification: DHL Express",
            "body": "Your package is on its way! Open the attached tracking document to see delivery details. Enable editing if prompted.",
            "subtype": "malware_shipping",
        },
        {
            "subject": "Voicemail from unknown caller",
            "body": "You have 1 new voicemail. Download the attached audio file to listen. Note: .exe format required for playback.",
            "subtype": "malware_voicemail",
        },
        {
            "subject": "Court summons - Immediate action required",
            "body": "You are hereby summoned to appear in court. Open the attached document for case details. Failure to respond may result in arrest.",
            "subtype": "malware_legal",
        },
        {
            "subject": "Failed delivery attempt - UPS",
            "body": "We attempted to deliver your package but no one was home. Print the attached label to pick up your package.",
            "subtype": "malware_shipping",
        },
    ]

    # Callback phishing (no links, phone-based)
    CALLBACK_PHISHING = [
        {
            "subject": "Suspicious activity on your account",
            "body": "We detected suspicious activity on your account. Call our security team immediately at 1-800-555-0123 to verify your identity.",
            "subtype": "callback_security",
        },
        {
            "subject": "Your order requires verification",
            "body": "Your order #12345 requires phone verification before shipping. Call 1-888-555-9999 with your order details.",
            "subtype": "callback_order",
        },
        {
            "subject": "Tax refund pending - IRS notice",
            "body": "You have a pending tax refund of $3,247.00. Call the IRS verification line at 1-800-555-8888 to claim.",
            "subtype": "callback_government",
        },
        {
            "subject": "Tech support alert",
            "body": "Critical security issue detected on your computer. Call Microsoft Support at 1-800-555-7777 immediately.",
            "subtype": "callback_tech_support",
        },
    ]

    def generate_dataset(
        self, total_samples: int = 500
    ) -> Tuple[List[str], List[str], List[int], List[str]]:
        """Generate a balanced dataset with diverse phishing types."""
        subjects = []
        bodies = []
        labels = []  # 0 = legitimate, 1 = phishing
        subtypes = []

        # Calculate samples per category
        legit_count = total_samples // 3
        phish_count = total_samples - legit_count
        phish_per_type = phish_count // 5

        # Generate legitimate emails
        for _ in range(legit_count):
            template = random.choice(self.LEGITIMATE_TEMPLATES)
            subjects.append(self._add_variation(template["subject"]))
            bodies.append(self._add_variation(template["body"]))
            labels.append(0)
            subtypes.append("legitimate")

        # Generate credential phishing
        for _ in range(phish_per_type):
            template = random.choice(self.CREDENTIAL_PHISHING)
            subjects.append(self._add_variation(template["subject"]))
            bodies.append(self._add_variation(template["body"]))
            labels.append(1)
            subtypes.append(template["subtype"])

        # Generate BEC
        for _ in range(phish_per_type):
            template = random.choice(self.BEC_PHISHING)
            subjects.append(self._add_variation(template["subject"]))
            bodies.append(self._add_variation(template["body"]))
            labels.append(1)
            subtypes.append(template["subtype"])

        # Generate spear phishing
        for _ in range(phish_per_type):
            template = random.choice(self.SPEAR_PHISHING)
            subjects.append(self._add_variation(template["subject"]))
            bodies.append(self._add_variation(template["body"]))
            labels.append(1)
            subtypes.append(template["subtype"])

        # Generate malware delivery
        for _ in range(phish_per_type):
            template = random.choice(self.MALWARE_PHISHING)
            subjects.append(self._add_variation(template["subject"]))
            bodies.append(self._add_variation(template["body"]))
            labels.append(1)
            subtypes.append(template["subtype"])

        # Generate callback phishing
        for _ in range(phish_count - 4 * phish_per_type):
            template = random.choice(self.CALLBACK_PHISHING)
            subjects.append(self._add_variation(template["subject"]))
            bodies.append(self._add_variation(template["body"]))
            labels.append(1)
            subtypes.append(template["subtype"])

        return subjects, bodies, labels, subtypes

    def _add_variation(self, text: str) -> str:
        """Add slight variations to text."""
        variations = [
            lambda t: t,
            lambda t: t.upper() if random.random() < 0.1 else t,
            lambda t: t
            + " "
            + random.choice(["Please respond ASAP.", "Thank you.", "Best regards.", ""]),
            lambda t: t.replace(".", "!") if random.random() < 0.2 else t,
        ]
        return random.choice(variations)(text)


# Generate comprehensive dataset
generator = PhishingDataGenerator()
subjects, bodies, labels, subtypes = generator.generate_dataset(total_samples=500)

# Combine into full text
texts = [f"Subject: {s}\n\n{b}" for s, b in zip(subjects, bodies)]

# Add timestamps (simulating 30-day email collection period)
# Phishing campaigns cluster in waves; legitimate emails spread evenly
from datetime import datetime, timedelta
base_date = datetime(2024, 1, 1)
timestamps = []
for label in labels:
    if label == 1:  # Phishing - cluster in campaign waves
        campaign_day = random.choice([3, 7, 15, 22, 28])
        day_offset = campaign_day + random.randint(-2, 2)
    else:  # Legitimate - spread evenly
        day_offset = random.randint(0, 29)
    hour = random.randint(6, 22)
    timestamps.append(base_date + timedelta(days=day_offset, hours=hour, minutes=random.randint(0, 59)))

# Create DataFrame
df = pd.DataFrame({
    "text": texts, "subject": subjects, "body": bodies,
    "label": labels, "subtype": subtypes, "timestamp": timestamps
})
df = df.sort_values("timestamp").reset_index(drop=True)

print(f"Dataset size: {len(df)} samples")
print(f"Time range: {df['timestamp'].min().strftime('%Y-%m-%d')} to {df['timestamp'].max().strftime('%Y-%m-%d')}")
print(f"\nClass distribution:")
print(df["label"].value_counts().rename({0: "Legitimate", 1: "Phishing"}))
print(f"\nPhishing subtypes:")
print(df[df["label"] == 1]["subtype"].value_counts())

In [ ]:
# Enhanced Email Classification Distribution
# Shows both counts and percentages for clearer understanding

total_emails = len(df)
legit_count = (df["label"] == 0).sum()
phish_count = (df["label"] == 1).sum()
legit_pct = legit_count / total_emails * 100
phish_pct = phish_count / total_emails * 100

# Create side-by-side: donut chart + bar breakdown
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "bar"}]],
    subplot_titles=["Dataset Composition", "Phishing Attack Types"],
    column_widths=[0.4, 0.6],
)

# Donut chart - overall distribution
fig.add_trace(
    go.Pie(
        labels=["✅ Legitimate", "🚨 Phishing"],
        values=[legit_count, phish_count],
        hole=0.5,
        marker=dict(colors=["#2ecc71", "#e74c3c"]),
        textinfo="label+percent",
        textposition="outside",
        hovertemplate="<b>%{label}</b><br>Count: %{value}<br>%{percent}<extra></extra>",
    ),
    row=1, col=1
)

# Bar chart - phishing subtypes breakdown
subtype_counts = df[df["label"] == 1]["subtype"].value_counts()
# Clean up subtype names for display
display_names = {
    "credential": "🔐 Credential Theft",
    "bec_ceo_fraud": "👔 CEO Fraud (BEC)",
    "bec_invoice_fraud": "📄 Invoice Fraud",
    "bec_payroll_diversion": "💰 Payroll Diversion",
    "bec_gift_card": "🎁 Gift Card Scam",
    "spear_personalized": "🎯 Spear Phishing",
    "spear_linkedin": "💼 LinkedIn Lure",
    "malware_invoice": "📎 Malware (Invoice)",
    "malware_job": "📝 Malware (Job App)",
    "malware_shipping": "📦 Malware (Shipping)",
    "malware_voicemail": "🎤 Malware (Voicemail)",
    "malware_legal": "⚖️ Malware (Legal)",
    "callback_security": "📞 Callback (Security)",
    "callback_order": "📞 Callback (Order)",
    "callback_government": "📞 Callback (Gov)",
    "callback_tech_support": "📞 Callback (Tech)",
}

fig.add_trace(
    go.Bar(
        x=[display_names.get(s, s) for s in subtype_counts.index],
        y=subtype_counts.values,
        marker_color="#e74c3c",
        marker_line_color="#c0392b",
        marker_line_width=1,
        text=subtype_counts.values,
        textposition="outside",
        hovertemplate="<b>%{x}</b><br>Count: %{y}<extra></extra>",
    ),
    row=1, col=2
)

fig.update_layout(
    title=dict(
        text="📧 Email Dataset Overview<br><sup>Understanding what we're classifying</sup>",
        font=dict(size=18),
    ),
    template=PLOTLY_TEMPLATE,
    height=450,
    width=1000,
    showlegend=False,
    annotations=[
        dict(
            text=f"<b>{total_emails}</b><br>emails",
            x=0.18, y=0.5,
            font=dict(size=16),
            showarrow=False,
            xref="paper", yref="paper",
        )
    ],
)
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)

fig.show()

# Summary stats
print(f"\n📊 Dataset Summary:")
print(f"   Total emails: {total_emails}")
print(f"   ✅ Legitimate: {legit_count} ({legit_pct:.1f}%)")
print(f"   🚨 Phishing: {phish_count} ({phish_pct:.1f}%)")
print(f"\n   Phishing types: {len(subtype_counts)} distinct attack patterns")

In [ ]:
# Phishing Campaign Timeline - visualize when attacks cluster
df['date'] = df['timestamp'].dt.date
daily_counts = df.groupby(['date', 'label']).size().unstack(fill_value=0)
daily_counts.columns = ['Legitimate', 'Phishing']
daily_counts = daily_counts.reset_index()
daily_counts['date'] = pd.to_datetime(daily_counts['date'])

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=['Daily Email Volume by Type', 'Cumulative Phishing Campaign Activity'],
    vertical_spacing=0.15,
)

# Daily volume
fig.add_trace(
    go.Bar(x=daily_counts['date'], y=daily_counts['Legitimate'], name='Legitimate', marker_color='#2ecc71'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=daily_counts['date'], y=daily_counts['Phishing'], name='Phishing', marker_color='#e74c3c'),
    row=1, col=1
)

# Cumulative phishing (shows campaign waves)
daily_counts['cumulative_phishing'] = daily_counts['Phishing'].cumsum()
fig.add_trace(
    go.Scatter(
        x=daily_counts['date'], y=daily_counts['cumulative_phishing'],
        name='Cumulative Phishing', mode='lines+markers',
        line=dict(color='#e74c3c', width=3),
        fill='tozeroy', fillcolor='rgba(231,76,60,0.2)'
    ),
    row=2, col=1
)

fig.update_layout(
    title='📅 Phishing Campaign Timeline Analysis',
    template=PLOTLY_TEMPLATE,
    height=500,
    barmode='group',
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=1.02),
)
fig.update_xaxes(title_text='Date', row=1, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text='Email Count', row=1, col=1)
fig.update_yaxes(title_text='Total Phishing Emails', row=2, col=1)
fig.show()

# Identify campaign peaks
peak_days = daily_counts.nlargest(3, 'Phishing')
print('🚨 Top 3 Phishing Campaign Days:')
for _, row in peak_days.iterrows():
    print(f"   {row['date'].strftime('%Y-%m-%d')}: {int(row['Phishing'])} phishing emails")


## 2. Feature Extraction with TF-IDF

### 🤔 The Problem: Computers Don't Understand Words

Machine learning models need **numbers**, not text. How do we turn emails into numbers?

**Naive approach: Just count words?**
```
Email: "Urgent! Click here now! Act immediately!"
→ "urgent": 1, "click": 1, "here": 1, "now": 1, "act": 1, "immediately": 1
```

**Problem**: Common words like "the", "is", "and" appear everywhere. They're not useful for classification!

### 📊 TF-IDF: Term Frequency × Inverse Document Frequency

TF-IDF solves this by making words **rarer = more important**:

```
┌───────────────────────────────────────────────────────────────────────────┐
│                         TF-IDF EXPLAINED                                  │
├───────────────────────────────────────────────────────────────────────────┤
│                                                                           │
│   TF (Term Frequency):                                                    │
│   "How often does this word appear in THIS email?"                        │
│                                                                           │
│   Email: "Urgent! Very urgent! Act now!"                                  │
│   TF("urgent") = 2/5 = 0.40  (appears twice in 5 words)                   │
│   TF("now")    = 1/5 = 0.20  (appears once)                               │
│                                                                           │
│   ─────────────────────────────────────────────────────────────────────── │
│                                                                           │
│   IDF (Inverse Document Frequency):                                       │
│   "How rare is this word across ALL emails?"                              │
│                                                                           │
│   Word       | Appears in | IDF Score | Interpretation                    │
│   ─────────────────────────────────────────────────────────               │
│   "the"      | 95% emails | Very LOW  | Common, not useful               │
│   "click"    | 20% emails | Medium    | Somewhat distinctive             │
│   "urgent"   | 5% emails  | HIGH      | Rare = likely important!         │
│   "wire"     | 1% emails  | Very HIGH | Very distinctive                 │
│                                                                           │
│   ─────────────────────────────────────────────────────────────────────── │
│                                                                           │
│   TF-IDF = TF × IDF                                                       │
│                                                                           │
│   High TF-IDF = word is frequent in THIS email BUT rare overall           │
│               = probably an important distinguishing word!                │
│                                                                           │
└───────────────────────────────────────────────────────────────────────────┘
```

### 🎯 Why TF-IDF Works for Phishing Detection

| Word | Appears in Legit Emails | Appears in Phishing | TF-IDF Signal |
|------|------------------------|---------------------|---------------|
| "meeting" | Very common | Rare | Low in phishing |
| "urgent" | Rare | Very common | **High in phishing** |
| "verify" | Rare | Very common | **High in phishing** |
| "attached" | Common | Common | Low everywhere |
| "suspended" | Very rare | Common in phishing | **Very high in phishing** |

**The magic**: TF-IDF automatically discovers that "urgent", "verify", and "suspended" are strong phishing indicators without us telling it!

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.3, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(
    max_features=1000, stop_words="english", ngram_range=(1, 2)  # Unigrams and bigrams
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Feature matrix shape: {X_train_tfidf.shape}")
print(f"\nTop 10 features:")
feature_names = vectorizer.get_feature_names_out()
print(feature_names[:10])

## 3. Train Random Forest Classifier

In [ ]:
# Train model
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

clf.fit(X_train_tfidf, y_train)
print("Model trained successfully!")

## 4. Evaluate Model Performance

In [ ]:
# Predictions
y_pred = clf.predict(X_test_tfidf)
y_prob = clf.predict_proba(X_test_tfidf)[:, 1]

# Classification report
print("Classification Report:")
print("=" * 50)
print(classification_report(y_test, y_pred, target_names=["Legitimate", "Phishing"]))

In [ ]:
# Interactive Confusion Matrix, ROC Curve, and Precision-Recall Curve
cm = confusion_matrix(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Precision-Recall curve (important for imbalanced data)
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)

# Create subplots: 1 row, 3 columns
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(
        "Confusion Matrix",
        f"ROC Curve (AUC = {roc_auc:.3f})",
        f"Precision-Recall (AP = {pr_auc:.3f})"
    ),
    specs=[[{"type": "heatmap"}, {"type": "scatter"}, {"type": "scatter"}]]
)

# Improved Confusion Matrix with quadrant labels
# Security interpretation:
#   TN (top-left): Correctly allowed legitimate emails
#   FP (top-right): False alarm - blocked legitimate email
#   FN (bottom-left): MISSED THREAT - phishing got through!
#   TP (bottom-right): Correctly blocked phishing
labels = ["Legitimate", "Phishing"]

# Create rich text with counts, percentages, and quadrant meaning
quadrant_labels = [
    ["✅ TN", "⚠️ FP"],  # Row 0 (Actual=Legitimate)
    ["🚨 FN", "✅ TP"]   # Row 1 (Actual=Phishing)
]
quadrant_meanings = [
    ["Correctly\nAllowed", "False\nAlarm"],
    ["MISSED\nTHREAT", "Correctly\nBlocked"]
]

cm_text = [
    [f"<b>{quadrant_labels[i][j]}</b><br>{cm[i][j]}<br>({cm[i][j]/cm.sum()*100:.1f}%)<br><sub>{quadrant_meanings[i][j]}</sub>"
     for j in range(2)]
    for i in range(2)
]

# Use diverging colorscale to highlight errors
# Custom colorscale: green for correct (TN, TP), red for errors (FP, FN)
fig.add_trace(
    go.Heatmap(
        z=cm,
        x=labels,
        y=labels,
        text=cm_text,
        texttemplate="%{text}",
        textfont={"size": 12},
        colorscale="Blues",
        showscale=False,
        hovertemplate=(
            "<b>Actual:</b> %{y}<br>"
            "<b>Predicted:</b> %{x}<br>"
            "<b>Count:</b> %{z}<br>"
            "<extra></extra>"
        ),
    ),
    row=1, col=1
)

# ROC Curve
fig.add_trace(
    go.Scatter(
        x=fpr, y=tpr,
        mode="lines",
        name=f"Classifier (AUC={roc_auc:.3f})",
        line=dict(color="#3498db", width=2),
        hovertemplate="False Alarm Rate: %{x:.1%}<br>Phishing Caught: %{y:.1%}<extra></extra>",
    ),
    row=1, col=2
)

# Diagonal reference line for ROC
fig.add_trace(
    go.Scatter(
        x=[0, 1], y=[0, 1],
        mode="lines",
        name="Random Guess",
        line=dict(color="#e74c3c", width=2, dash="dash"),
        showlegend=False,
    ),
    row=1, col=2
)

# Precision-Recall Curve with security context
# Recall = % of actual phishing caught (sensitivity)
# Precision = % of flagged emails that are actually phishing

fig.add_trace(
    go.Scatter(
        x=recall, y=precision,
        mode="lines",
        name=f"Classifier",
        line=dict(color="#9b59b6", width=3),
        fill="tozeroy",
        fillcolor="rgba(155, 89, 182, 0.15)",
        hovertemplate=(
            "<b>Operating Point</b><br>"
            "Catch Rate: %{x:.1%} of phishing<br>"
            "Accuracy: %{y:.1%} when flagging<br>"
            "<extra></extra>"
        ),
    ),
    row=1, col=3
)

# Baseline for PR curve (random classifier)
baseline = y_test.sum() / len(y_test)
fig.add_trace(
    go.Scatter(
        x=[0, 1], y=[baseline, baseline],
        mode="lines",
        name="Random Guess",
        line=dict(color="gray", width=2, dash="dash"),
        hovertemplate=f"Random: {baseline:.1%} precision<extra></extra>",
        showlegend=False,
    ),
    row=1, col=3
)

# Add "sweet spot" region annotation
fig.add_annotation(
    x=0.85, y=0.85,
    text="🎯 Ideal:<br>High both",
    showarrow=False,
    font=dict(size=9, color="#27ae60"),
    bgcolor="rgba(39,174,96,0.1)",
    bordercolor="#27ae60",
    borderwidth=1,
    row=1, col=3
)

fig.update_layout(
    template=PLOTLY_TEMPLATE,
    height=400,
    width=1200,
    showlegend=True,
    legend=dict(x=0.85, y=0.15),
)

fig.update_xaxes(title_text="Predicted", row=1, col=1)
fig.update_yaxes(title_text="Actual", row=1, col=1)
fig.update_xaxes(title_text="False Alarm Rate", row=1, col=2, range=[0, 1])
fig.update_yaxes(title_text="Detection Rate", row=1, col=2, range=[0, 1])

# Add interpretation guide for ROC
fig.add_annotation(
    x=0.6, y=0.3,
    text="⬆️ Higher is better<br>(more phishing caught)",
    showarrow=False,
    font=dict(size=10, color="gray"),
    row=1, col=2
)
fig.update_xaxes(title_text="Recall (Catch Rate)", row=1, col=3, range=[0, 1])
fig.update_yaxes(title_text="Precision (Flag Accuracy)", row=1, col=3, range=[0, 1])

fig.show()

# Print security-focused interpretation
print("\n📊 Security Metrics Explained:")
print("=" * 55)
print(f"\n🎯 ROC AUC: {roc_auc:.3f}")
print("   How well we separate phishing from legitimate")
print("   0.5 = random guess, 1.0 = perfect separation")
print(f"\n📈 PR AUC (Average Precision): {pr_auc:.3f}")
print("   Overall quality at different thresholds")
print(f"   Baseline (random): {baseline:.3f}")
print(f"\n💡 Precision-Recall Trade-off:")
print("   • High Recall = Catch more phishing (but more false alarms)")
print("   • High Precision = Fewer false alarms (but miss some phishing)")
print("   • Security often prioritizes Recall (don't miss threats!)")

## 5. Feature Importance Analysis

In [ ]:
# Enhanced Phishing Indicators - categorized and explained
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1][:20]
top_features = [feature_names[i] for i in indices]
top_importances = importances[indices]

# Categorize features by phishing tactic
def categorize_feature(feature):
    """Assign category based on feature content."""
    feature_lower = feature.lower()

    # Urgency/pressure tactics
    if any(w in feature_lower for w in ["urgent", "immediately", "now", "asap", "act", "expires", "limited"]):
        return "🔴 Urgency", "#e74c3c"
    # Security/account threats
    elif any(w in feature_lower for w in ["account", "password", "verify", "secure", "compromised", "suspended", "locked"]):
        return "🔒 Security Threat", "#9b59b6"
    # Financial/monetary lures
    elif any(w in feature_lower for w in ["wire", "payment", "invoice", "bank", "money", "transfer", "gift"]):
        return "💰 Financial", "#f39c12"
    # Action requests
    elif any(w in feature_lower for w in ["click", "open", "download", "enable", "sign", "confirm"]):
        return "👆 Action Request", "#3498db"
    # Authority/impersonation
    elif any(w in feature_lower for w in ["ceo", "director", "hr", "it", "support", "admin"]):
        return "👔 Authority", "#1abc9c"
    else:
        return "📝 Content", "#95a5a6"

# Build enhanced dataframe
importance_df = pd.DataFrame({
    "feature": top_features,
    "importance": top_importances,
})
importance_df["category"], importance_df["color"] = zip(*importance_df["feature"].apply(categorize_feature))

# Create visualization
fig = go.Figure()

# Add bars with category colors
for category in importance_df["category"].unique():
    cat_data = importance_df[importance_df["category"] == category]
    fig.add_trace(
        go.Bar(
            y=cat_data["feature"],
            x=cat_data["importance"],
            orientation="h",
            name=category,
            marker_color=cat_data["color"].iloc[0],
            hovertemplate=(
                "<b>%{y}</b><br>"
                "Importance: %{x:.4f}<br>"
                f"Category: {category}<br>"
                "<extra></extra>"
            ),
        )
    )

fig.update_layout(
    title=dict(
        text="🎣 Top Phishing Indicators by Category<br><sup>What words/phrases signal a phishing attack?</sup>",
        font=dict(size=16),
    ),
    template=PLOTLY_TEMPLATE,
    yaxis=dict(categoryorder="total ascending"),
    xaxis_title="Feature Importance (higher = stronger signal)",
    yaxis_title="Feature (word or phrase)",
    height=550,
    width=850,
    legend=dict(
        title="Phishing Tactic",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
    barmode="overlay",
)

fig.show()

# Print interpretation guide
print("\n🎣 Phishing Indicator Categories:")
print("=" * 50)
print("  🔴 Urgency    - Creates pressure to act fast")
print("  🔒 Security   - Fake account/security warnings")
print("  💰 Financial  - Money, payments, wire transfers")
print("  👆 Action     - Click, download, enable macros")
print("  👔 Authority  - Impersonates executives/IT")
print("  📝 Content    - General suspicious content")
print("\n💡 Tip: Attackers combine multiple tactics for effectiveness!")

## 6. Test with New Emails

In [ ]:
# Test emails
test_emails = [
    "URGENT: Your account will be closed. Click here immediately!",
    "Hi, let's catch up over coffee next week. How's Tuesday?",
    "You've won a free iPhone! Claim now before it expires!",
]

from IPython.display import display, Markdown


def classify_email(email_text: str) -> tuple[str, float]:
    """Classify a single email as phishing or legitimate.

    Args:
        email_text: The email text to classify

    Returns:
        Tuple of (result label, confidence percentage)
    """
    # Transform email text using the trained vectorizer
    email_tfidf = vectorizer.transform([email_text])

    # Get prediction and probability
    prediction = clf.predict(email_tfidf)[0]
    probabilities = clf.predict_proba(email_tfidf)[0]

    # Get confidence for the predicted class
    confidence = probabilities[prediction] * 100
    result = "PHISHING" if prediction == 1 else "LEGITIMATE"

    return result, confidence


# Build markdown output
md_output = "## 📧 Email Classification Results\n\n"
md_output += "| Status | Confidence | Email Preview |\n"
md_output += "|--------|------------|---------------|\n"

for email in test_emails:
    result, confidence = classify_email(email)
    icon = "🚨 **PHISHING**" if result == "PHISHING" else "✅ Legitimate"
    preview = email[:45] + "..." if len(email) > 45 else email
    md_output += f"| {icon} | {confidence:.1f}% | {preview} |\n"

md_output += "\n---\n"
md_output += "\n**Legend**: 🚨 = Phishing detected, ✅ = Legitimate email"

display(Markdown(md_output))

## Summary

In this lab, we built a phishing email classifier using:
- **TF-IDF vectorization** to convert email text to numerical features
- **Random Forest classifier** for robust classification
- **Evaluation metrics** including precision, recall, F1, and ROC-AUC

### Key Phishing Indicators Identified:
- Urgency words ("urgent", "immediately", "act now")
- Financial incentives ("won", "prize", "free")
- Security threats ("compromised", "suspended", "verify")

### Next Steps:
1. Add more training data
2. Try deep learning models (BERT, RoBERTa)
3. Add header analysis features
4. Integrate with email gateway